<a href="https://colab.research.google.com/github/Siddhant24/board-snapper/blob/master/Board_snapper_Eshan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#!rm -r board-snapper
#!git clone https://github.com/Siddhant24/board-snapper


fatal: destination path 'board-snapper' already exists and is not an empty directory.


In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import math
import os
import keras
from keras.utils import generic_utils
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, Conv3D, TimeDistributed, Dropout, Flatten, MaxPooling2D, MaxPool3D, Reshape, LSTM, Bidirectional
from keras import regularizers
import h5py
from keras.optimizers import Adam
from sklearn.metrics import classification_report



Using TensorFlow backend.


In [0]:
def load_data(x_folder,y_folder,start,end):
  x_data=[];
  y_data=None;
    
  for i in range(start,end):
    src='lec'+str(i);
    image_folder=x_folder+src;
    for j in range(1, len(os.listdir(image_folder))+1):
      img = 'slide'+str(j)+'.jpg'
      image=cv2.resize(cv2.imread(os.path.join(image_folder,img)),(224,224))/255;
      x_data.append(image);

    label=np.loadtxt((y_folder+src+'.csv'),dtype=int);
    y_data = np.concatenate([y_data, label]) if y_data is not None else label;
		
    print(src+"loaded");
    
  y_data=np.reshape(y_data,(-1,1)).astype(int);
  return np.array(x_data),np.array(y_data);

#resize to /255
#

In [0]:
def format_train_data(in_x,in_y,timestep, repetition = 0.5):
  n_samples=in_x.shape[0]
  n_batches=int((n_samples-timestep)/((1-repetition)*timestep))+1
  dim=(n_batches,timestep,in_x.shape[1],in_x.shape[2],in_x.shape[3])
  
  x_train=np.zeros(dim);
  y_train=np.zeros((n_batches,timestep,1)).astype(float);
  sample_wts=np.zeros((n_batches,timestep));

  index=0;
  for i in range (n_batches):
    x_train[i]=in_x[index:index+timestep];
    y_train[i]=in_y[index:index+timestep];
    sample_wts[i,:]=y_train[i,:][:,0]*20+1;
    index=int(index+(1-repetition)*timestep);	

  return x_train,y_train,sample_wts;



In [0]:
def revert(y_p,timestep, repetition = 0.5):
	length=y_p.shape[0]
	output=np.zeros(int((length-1)*(1-repetition)*timestep+timestep));
	index=0;
	for i in range(length):
		output[index:index+timestep]=np.maximum(output[index:index+timestep],y_p[i][:,0]);
		index=int(index+(1-repetition)*timestep);	
	return output;






In [5]:
def getmodel(timestep):
	model=Sequential();
	model.add(TimeDistributed(Conv2D(16, (3, 3), activation='relu',padding='same'),input_shape= (timestep,224, 224, 3)));
	model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))));
	model.add(TimeDistributed(Conv2D(32, (3, 3),activation='relu',padding='same',kernel_regularizer = regularizers.l2('0.2'))));
	model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
	model.add(TimeDistributed(Conv2D(64, (3, 3),activation='relu',padding='same',kernel_regularizer = regularizers.l2('0.2'))));
	model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
	model.add(TimeDistributed(Conv2D(32, (3, 3),activation='relu',padding='same',kernel_regularizer = regularizers.l2('0.2'))));
	model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))));
	model.add(TimeDistributed(Flatten()));
	model.add(Bidirectional(LSTM(128,activation='tanh',return_sequences=True, kernel_regularizer=regularizers.l2('0.2')),merge_mode='concat'));
	model.add(TimeDistributed(Dense(100,activation='relu')));
	model.add(TimeDistributed(Dense(1,activation='sigmoid')))
	model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),sample_weight_mode='temporal');
	model.summary();
	return model;

getmodel(10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 10, 224, 224, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 112, 112, 16)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 10, 112, 112, 32)  4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 56, 56, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 56, 56, 64)    18496     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 10, 28, 28, 64)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 10, 28, 28, 32)    18464     
__________

In [6]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [0]:
x_folder='board-snapper/slides/';
y_folder='board-snapper/labels/';

In [17]:
timestep=16;
batch_size=10;

lrcn=getmodel(timestep);

%cd ..
%cd board-snapper

#lrcn.save_weights('lrcn_weights trained 21-50.h5')

for i in range(21, 51):
  lrcn.load_weights('lrcn_weights trained 21-50.h5')
  x_data,y_data=load_data(x_folder,y_folder,i, i+1);
  x_train, y_train, sample_wts= format_train_data(x_data,y_data,timestep);
  
  n_epochs= 10;
  lrcn.fit(x_train,y_train, batch_size = batch_size,epochs=n_epochs,sample_weight=sample_wts);
  lrcn.save_weights('lrcn_weights trained 21-50.h5')


#lrcn.save('board-snapper/lstm_'+str(timestep)+'.h5');
#lrcn.save_weights('board-snapper/lstm_weights_'+str(timestep)+'.h5')

lrcn.save('1')




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_45 (TimeDis (None, 16, 224, 224, 16)  448       
_________________________________________________________________
time_distributed_46 (TimeDis (None, 16, 112, 112, 16)  0         
_________________________________________________________________
time_distributed_47 (TimeDis (None, 16, 112, 112, 32)  4640      
_________________________________________________________________
time_distributed_48 (TimeDis (None, 16, 56, 56, 32)    0         
_________________________________________________________________
time_distributed_49 (TimeDis (None, 16, 56, 56, 64)    18496     
_________________________________________________________________
time_distributed_50 (TimeDis (None, 16, 28, 28, 64)    0         
_________________________________________________________________
time_distributed_51 (TimeDis (None, 16, 28, 28, 32)    18464     
__________

In [0]:
lrcn.save('1')
lrcn = keras.models.load_model('1')

In [0]:
lrcn.save_weights('1.h5')
lrcn.load_weights('1.h5')

In [33]:

!git add --al
!git commit -m "Saving_Weights"
!git push


[master 07830c3e] Saving_Weights
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 lrcn_weights trained 21-50.h5
fatal: could not read Username for 'https://github.com': No such device or address


 20lstm_weights.h5      convLstmbatch.py		 lstm.py
'Board Snapper.ipynb'   labels				 lstm_weights.h5
 Board_Snapper.ipynb   'lrcn_weights trained 21-50.h5'	 slides


In [0]:
!git config --global user.email "eshan.balachandar@gmail.com"
!git config --global user.name "MisterMediocre"

In [0]:
lec = 21
x_t,y_t=load_data(x_folder,y_folder,lec,lec+1);
x_test,y_test,y_sample_wt=format_train_data(x_t,y_t,timestep);



In [0]:
y_pred = lrcn.predict(x_test,batch_size=4);
y_form = np.reshape(revert(y_pred,timestep),(-1,1));




In [35]:
y_p=((y_form>0.8)*1);
target_names = ['non_key', 'key']
print(classification_report(y_t, y_p, target_names=target_names));



             precision    recall  f1-score   support

    non_key       1.00      1.00      1.00       842
        key       1.00      1.00      1.00        22

avg / total       1.00      1.00      1.00       864

